# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

In this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this project.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

The next step is to import the constituents of the S&P 500.

These constituents change over time, so in an ideal world we would connect directly to the index provider and pull their real-time constituents on a regular basis.

The index provider for real-time data are paid, so here we are using a static version which is free. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](http://nickmccullum.com/algorithmic-trading-python/sp_500_stocks.csv). This file is moved into the same folder where this python file exists.

Now it's time to import these stocks to the Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import the IEX Cloud API token. This is the data provider that we will use in this project.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to the local Git repository. We will use a sandbox API token in this project, which means that the data we will use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download the `secrets.py` file. The file is moved into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make  our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2451115778587,
 'week52high': 156,
 'week52low': 89.71,
 'week52highSplitAdjustOnly': 156,
 'week52lowSplitAdjustOnly': 89.64,
 'week52change': 0.608582372739264,
 'sharesOutstanding': 17265998347,
 'float': 0,
 'avg10Volume': 100079776,
 'avg30Volume': 91587413,
 'day200MovingAvg': 135.19,
 'day50MovingAvg': 140.95,
 'employees': 150462,
 'ttmEPS': 4.61,
 'ttmDividendRate': 0.846467395040993,
 'dividendYield': 0.00579154462794497,
 'nextDividendDate': '',
 'exDividendDate': '2021-05-04',
 'nextEarningsDate': '2021-07-19',
 'peRatio': 32.79715657269566,
 'beta': 1.5744994523733933,
 'maxChangePercent': 55.45549694160302,
 'year5ChangePercent': 5.435597436380124,
 'year2ChangePercent': 1.933091952121493,
 'year1ChangePercent': 0.5225217508805011,
 'ytdChangePercent': 0.11042808637396215,
 'month6ChangePercent': 0.06013377267196409,
 'month3ChangePercent': 0.11736214191705714,
 'month1ChangePercent': 0.09915703880073556,
 'day30ChangePercent': 0

## Parsing Our API Call

This API call has all the information we need. 
With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
data['year1ChangePercent']

0.5225217508805011

## Using Batch API Calls for good Performance

Batch API calls are one of the easiest ways to improve the performance of API calls.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give us discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this project from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,159.300,0.597075,N/A
1,AAL,21.360,0.876326,N/A
2,AAP,218.700,0.427503,N/A
3,AAPL,153.851,0.533525,N/A
4,ABBV,120.410,0.265584,N/A
...,...,...,...,...
500,YUM,123.900,0.311409,N/A
501,ZBH,166.500,0.207681,N/A
502,ZBRA,570.960,0.968147,N/A
503,ZION,51.748,0.576691,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building, seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,80.770,3.04859,N/A
1,TPR,40.700,1.99198,N/A
2,FCX,36.425,1.63327,N/A
3,SIVB,590.278,1.62297,N/A
4,DVN,25.675,1.54592,N/A
5,COF,163.070,1.54469,N/A
6,DFS,125.520,1.51031,N/A
7,IVZ,25.640,1.49173,N/A
8,MGM,39.880,1.45667,N/A
9,KSS,50.500,1.37747,N/A


## Calculating the Number of Shares to Buy

We now need to calculate the number of shares we need to buy.

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()

Enter the value of your portfolio:10000000


In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/home/shreyansh/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,80.770,3.04859,2476
1,TPR,40.700,1.99198,4914
2,FCX,36.425,1.63327,5490
3,SIVB,590.278,1.62297,338
4,DVN,25.675,1.54592,7789
5,COF,163.070,1.54469,1226
6,DFS,125.520,1.51031,1593
7,IVZ,25.640,1.49173,7800
8,MGM,39.880,1.45667,5015
9,KSS,50.500,1.37747,3960


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame(hqm_dataframe). 'hqm' stands for `high-quality momentum`.

In [11]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #print(data)
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,156.880,N/A,0.585916,N/A,0.198281,N/A,0.123883,N/A,0.0274024,N/A,N/A
1,AAL,21.680,N/A,0.903465,N/A,0.348193,N/A,0.0570017,N/A,-0.0450793,N/A,N/A
2,AAP,223.400,N/A,0.434426,N/A,0.303062,N/A,0.0576427,N/A,0.052848,N/A,N/A
3,AAPL,153.088,N/A,0.528317,N/A,0.0607005,N/A,0.117203,N/A,0.0967251,N/A,N/A
4,ABBV,122.140,N/A,0.269336,N/A,0.0863771,N/A,0.0825833,N/A,0.0374978,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.600,N/A,0.310314,N/A,0.134836,N/A,0.0252114,N/A,0.0241054,N/A,N/A
501,ZBH,167.820,N/A,0.205559,N/A,-0.0117914,N/A,-0.1093,N/A,-0.0466628,N/A,N/A
502,ZBRA,564.070,N/A,0.956206,N/A,0.324077,N/A,0.101425,N/A,0.0609908,N/A,N/A
503,ZION,51.450,N/A,0.567395,N/A,0.0616162,N/A,-0.0383094,N/A,-0.0482711,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [12]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]
for row in hqm_dataframe.index:
    for time_period in time_periods:   
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0

for row in hqm_dataframe.index:
    for time_period in time_periods:
        percentile_col = f'{time_period} Return Percentile'
        change_col = f'{time_period} Price Return'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe[change_col][row])/100

#Print each percentile score to make sure it was calculated properly
#for time_period in time_periods:
#    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,156.880,N/A,0.585916,0.734653,0.198281,0.659406,0.123883,0.823762,0.0274024,0.645545,N/A
1,AAL,21.680,N/A,0.903465,0.90297,0.348193,0.90495,0.0570017,0.657426,-0.0450793,0.176238,N/A
2,AAP,223.400,N/A,0.434426,0.576238,0.303062,0.857426,0.0576427,0.661386,0.052848,0.817822,N/A
3,AAPL,153.088,N/A,0.528317,0.687129,0.0607005,0.265347,0.117203,0.811881,0.0967251,0.944554,N/A
4,ABBV,122.140,N/A,0.269336,0.340594,0.0863771,0.344554,0.0825833,0.720792,0.0374978,0.706931,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.600,N/A,0.310314,0.39802,0.134836,0.471287,0.0252114,0.526733,0.0241054,0.625743,N/A
501,ZBH,167.820,N/A,0.205559,0.283168,-0.0117914,0.122772,-0.1093,0.0653465,-0.0466628,0.164356,N/A
502,ZBRA,564.070,N/A,0.956206,0.918812,0.324077,0.875248,0.101425,0.784158,0.0609908,0.855446,N/A
503,ZION,51.450,N/A,0.567395,0.710891,0.0616162,0.267327,-0.0383094,0.265347,-0.0482711,0.152475,N/A


Here we have used the if statement because few rows in the data provided by API has null value instead of float, which would give us an error, if we don't convert it to float.

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [13]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [14]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FTNT,281.230,N/A,1.05339,0.950495,0.796315,1,0.359673,0.99604,0.119266,0.982178,0.982178
1,HCA,252.960,N/A,1.04498,0.948515,0.48777,0.980198,0.228865,0.960396,0.183464,0.994059,0.970792
2,LB,80.860,N/A,2.95041,1,0.697315,0.99604,0.163591,0.89703,0.108263,0.962376,0.963861
3,IT,264.898,N/A,1.06672,0.952475,0.620861,0.994059,0.32173,0.992079,0.0727129,0.89703,0.958911
4,IDXX,706.840,N/A,0.90072,0.90099,0.390732,0.940594,0.254066,0.978218,0.117685,0.978218,0.949505
5,TGT,268.190,N/A,1.1236,0.958416,0.355998,0.908911,0.265771,0.982178,0.0847339,0.920792,0.942574
6,INTU,553.540,N/A,0.776935,0.861386,0.409151,0.950495,0.291042,0.990099,0.0957279,0.940594,0.935644
7,ORCL,91.030,N/A,0.647452,0.79802,0.533738,0.990099,0.21992,0.954455,0.163199,0.990099,0.933168
8,MS,96.310,N/A,0.919073,0.908911,0.303578,0.861386,0.226008,0.958416,0.118453,0.980198,0.927228
9,COF,161.450,N/A,1.5797,0.994059,0.557027,0.992079,0.217423,0.950495,0.043554,0.760396,0.924257


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [15]:
portfolio_input()

Enter the value of your portfolio:10000000


In [16]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

/home/shreyansh/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FTNT,281.230,711,1.05339,0.950495,0.796315,1,0.359673,0.99604,0.119266,0.982178,0.982178
1,HCA,252.960,790,1.04498,0.948515,0.48777,0.980198,0.228865,0.960396,0.183464,0.994059,0.970792
2,LB,80.860,2473,2.95041,1,0.697315,0.99604,0.163591,0.89703,0.108263,0.962376,0.963861
3,IT,264.898,755,1.06672,0.952475,0.620861,0.994059,0.32173,0.992079,0.0727129,0.89703,0.958911
4,IDXX,706.840,282,0.90072,0.90099,0.390732,0.940594,0.254066,0.978218,0.117685,0.978218,0.949505
5,TGT,268.190,745,1.1236,0.958416,0.355998,0.908911,0.265771,0.982178,0.0847339,0.920792,0.942574
6,INTU,553.540,361,0.776935,0.861386,0.409151,0.950495,0.291042,0.990099,0.0957279,0.940594,0.935644
7,ORCL,91.030,2197,0.647452,0.79802,0.533738,0.990099,0.21992,0.954455,0.163199,0.990099,0.933168
8,MS,96.310,2076,0.919073,0.908911,0.303578,0.861386,0.226008,0.958416,0.118453,0.980198,0.927228
9,COF,161.450,1238,1.5797,0.994059,0.557027,0.992079,0.217423,0.950495,0.043554,0.760396,0.924257


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users.

In [17]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* Integer format for the number of shares to purchase
* Percentage format for others



In [18]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [19]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [20]:
writer.save()